### Importy i Ustawienia

In [1]:
from openai import OpenAI
import pandas as pd
import pickle
import openai
from tqdm import tqdm  # For progress bar, useful for large datasets
import ourfuncs as funk

PICKLE_PATH         = 'Saved/textdata.pkl'
EMBEDDING_MODEL     = "text-embedding-3-small"
SAVE_EMBDEDED_PKL   = f'Saved/textembed ({EMBEDDING_MODEL}).pkl'
SAVE_EMBDEDED_CSV   = f'Saved/textembed ({EMBEDDING_MODEL}).csv'

In [2]:
client  = OpenAI(api_key=funk.apikey())
data    = funk.load_data(PICKLE_PATH)
df      = pd.DataFrame(data, columns=['Document', 'Page', 'Block', "WordCount", 'Text'])
df


,Document,Page,Block,WordCount,Text
0,DDDD & DD.pdf,1,1,24,DDDD & DD. Dynamicznie Dedukowane Drzewa Decyz...
1,DDDD & DD.pdf,2,1,125,"W branży jubilerskiej, gdzie decyzje o zarządz..."
2,DDDD & DD.pdf,2,2,25,"omówienia, za przykład weźmy wybór następujący..."
3,DDDD & DD.pdf,2,3,54,"- Poziom 3 ‘Kamienie’ np. Cyrkonia, Brylant, G..."
4,DDDD & DD.pdf,3,1,119,Python W zależności od potrzeb obliczone już w...
...,...,...,...,...,...
67,Ustawa.pdf,14,1,27,©Kancelaria Sejmu s. 14/15 2023-09-21 w art 18...
68,Ustawa.pdf,14,2,54,umożliwiających spełnienie wymogów przewidzian...
69,Ustawa.pdf,14,3,74,"przewidzianym w art. 18 ust 1, w braku odmienn..."
70,Ustawa.pdf,14,4,78,umożliwiającą najpóźniej od dnia 1 październik...


### Embedding

In [3]:
def get_embedding(text):
    response = client.embeddings.create(
        input=text,
        model=EMBEDDING_MODEL
    )
    # Assuming the response structure fits this format; adjust according to actual API response
    return response.data[0].embedding

embeddings = []
# zrobienie embeddingu na każdy [Text]
for text in tqdm(df['Text'], desc="Generating embeddings"):
    embeddings.append(get_embedding(text))

# Dołączenie embeddingów do dataframe
df['Embedding'] = embeddings

# Zapis do nowego pickle (.pkl) oraz csv (do sprawdzania oczami)
funk.save_data(df, SAVE_EMBDEDED_PKL)
with open(SAVE_EMBDEDED_CSV, mode='w', newline='', encoding='utf-8-sig') as file:
    df.to_csv(file, index=False)

Generating embeddings: 100%|██████████| 72/72 [00:20<00:00,  3.46it/s]

Data saved to Saved/textembed (text-embedding-3-small).pkl


### Sprawdzenie

In [4]:
df = funk.load_data(SAVE_EMBDEDED_PKL)
df

,Document,Page,Block,WordCount,Text,Embedding
0,DDDD & DD.pdf,1,1,24,DDDD & DD. Dynamicznie Dedukowane Drzewa Decyz...,"[-0.011432180181145668, 0.029889998957514763, ..."
1,DDDD & DD.pdf,2,1,125,"W branży jubilerskiej, gdzie decyzje o zarządz...","[-0.060658082365989685, 0.01965409703552723, 0..."
2,DDDD & DD.pdf,2,2,25,"omówienia, za przykład weźmy wybór następujący...","[-0.0008431045571342111, 0.057511720806360245,..."
3,DDDD & DD.pdf,2,3,54,"- Poziom 3 ‘Kamienie’ np. Cyrkonia, Brylant, G...","[-0.05206809565424919, 0.045071445405483246, 0..."
4,DDDD & DD.pdf,3,1,119,Python W zależności od potrzeb obliczone już w...,"[-0.0054521868005394936, 0.04055621474981308, ..."
...,...,...,...,...,...,...
67,Ustawa.pdf,14,1,27,©Kancelaria Sejmu s. 14/15 2023-09-21 w art 18...,"[-0.04066246375441551, 0.08031929284334183, 0...."
68,Ustawa.pdf,14,2,54,umożliwiających spełnienie wymogów przewidzian...,"[0.03570476546883583, 0.051144663244485855, 0...."
69,Ustawa.pdf,14,3,74,"przewidzianym w art. 18 ust 1, w braku odmienn...","[-0.02101951092481613, 0.09580406546592712, 0...."
70,Ustawa.pdf,14,4,78,umożliwiającą najpóźniej od dnia 1 październik...,"[0.029946209862828255, 0.05419602245092392, 0...."
